In [1]:
# Display page w/ 90% margins
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# PGE Propensity Data Transform

In [2]:
# Import dependencies
# ... pandas to work with data
import pandas as pd
# ... glob to read multiple files from directory
import glob

### Create dataframe

In [3]:
# Create dataframe from pre-selected path for .csv files
path = 'C:\\Users\\Robert.Jones\\Documents\\LiHEAP\\customer_data'
csv_files = glob.glob(path + '/*.csv')
# Files included from Chris: "I uploaded the files from the FTP site to PGE FTP Data" 
# ... 60407_CCES_LIHEAP_PASTDUE_Feb_2022.xlsx to 60407_CCES_LIHEAP_PASTDUE_Jun_2022.xlsx
df_list = (pd.read_csv(file) for file in csv_files)
# Bring all files together in one concatenated dataframe
df = pd.concat(df_list)
# Print top 3 rows
df.head(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,PREM_ID,ACCT_ID,CUSTOMER_NAME,HOME_PHONE,BUSINESS_PHONE,CELL_PHONE,...,MAIL_STATE,MAIL_ZIP,CARE,TOTAL_DUE,PAST_DUE,LAST_BILL_DT,CAREpropensity,AMP Enrolled,ESA Treated,date
0,0,0,0,0,8831363511,2950107861,"O*****,P********",NaN,NaN,(831) 234-9764,...,CA,950192826,1,398.03,169.16,44673.0,2,N,N,2022-04-01
1,1,1,1,1,42734571,1146646056,"R*****,S*****",NaN,NaN,(831) 537-5212,...,CA,950234449,1,987.83,885.79,44678.0,2,N,N,2022-04-01
2,2,2,2,2,8344321417,2029206695,"H********,N*****",(415) 596-7597,NaN,(415) 559-1817,...,CA,941122314,1,67.01,0.07,44680.0,4,N,N,2022-04-01


### Remove Duplicates

In [4]:
# to check original # of rows
original_count = df.shape[0]
# drop dups on selected column names
df = df.drop_duplicates(subset=['ACCT_ID','CUSTOMER_NAME'],ignore_index=True)
# calculate and print dropped rows 
dropped = original_count - df.shape[0]
print(f'Duplicates dropped: {dropped}')

Duplicates dropped: 492658


### Split name on comma

In [5]:
df['first_name']= df.CUSTOMER_NAME.str.split(',',expand=True)[0]
df['last_name']= df.CUSTOMER_NAME.str.split(',',expand=True)[1]
print(df[['CUSTOMER_NAME','first_name', 'last_name']].head(5))

             CUSTOMER_NAME first_name        last_name
0         O*****,P********     O*****        P********
1            R*****,S*****     R*****           S*****
2         H********,N*****  H********           N*****
3    R****,M**** G********      R****  M**** G********
4  D******,M****** C******    D******  M****** C******


### Drop unnamed columns

In [6]:
initial = len(list(df))
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
df.head(5)
after_drop = len(list(df))
print(initial - after_drop," unnamed columns dropped")
df.head(3)

4  unnamed columns dropped


,PREM_ID,ACCT_ID,CUSTOMER_NAME,HOME_PHONE,BUSINESS_PHONE,CELL_PHONE,EMAIL,PREM_ADDRESS1,PREM_ADDRESS2,PREM_CITY,...,CARE,TOTAL_DUE,PAST_DUE,LAST_BILL_DT,CAREpropensity,AMP Enrolled,ESA Treated,date,first_name,last_name
0,8831363511,2950107861,"O*****,P********",NaN,NaN,(831) 234-9764,T*********@hotmail.com,113 SIDE AVE,NaN,FREEDOM,...,1,398.03,169.16,44673.0,2,N,N,2022-04-01,O*****,P********
1,42734571,1146646056,"R*****,S*****",NaN,NaN,(831) 537-5212,NaN,1371 WESTSIDE BLVD APT A,NaN,HOLLISTER,...,1,987.83,885.79,44678.0,2,N,N,2022-04-01,R*****,S*****
2,8344321417,2029206695,"H********,N*****",(415) 596-7597,NaN,(415) 559-1817,nelsoH********332@gmail.com,1168-1170 PLYMOUTH AVE - HSE METER,NaN,SAN FRANCISCO,...,1,67.01,0.07,44680.0,4,N,N,2022-04-01,H********,N*****


### Rearrange columns

In [7]:
# Generate list of columns
column_list = list(df)
# ... arrange columns
column_list = column_list[:3] + column_list[-2:] + column_list[5:]
# ... put them back into dataframe
df = df[column_list]
# drop last two duplicate items that we moved 
df = df.iloc[:, :-2]
df.head(3)

,PREM_ID,ACCT_ID,CUSTOMER_NAME,first_name,last_name,CELL_PHONE,EMAIL,PREM_ADDRESS1,PREM_ADDRESS2,PREM_CITY,...,MAIL_STATE,MAIL_ZIP,CARE,TOTAL_DUE,PAST_DUE,LAST_BILL_DT,CAREpropensity,AMP Enrolled,ESA Treated,date
0,8831363511,2950107861,"O*****,P********",O*****,P********,(831) 234-9764,T***********@hotmail.com,113 HILLSIDE AVE,NaN,FREEDOM,...,CA,950192826,1,398.03,169.16,44673.0,2,N,N,2022-04-01
1,42734571,1146646056,"R*****,S*****",R*****,S*****,(831) 537-5212,NaN,1371 WESTSIDE BLVD APT A,NaN,HOLLISTER,...,CA,950234449,1,987.83,885.79,44678.0,2,N,N,2022-04-01
2,8344321417,2029206695,"H********,N*****",H********,N*****,(415) 559-1817,nelsoH********332@gmail.com,1168-1170 PLYMOUTH AVE - HSE METER,NaN,SAN FRANCISCO,...,CA,941122314,1,67.01,0.07,44680.0,4,N,N,2022-04-01


### Sort Values by Date

In [8]:
# Sort by date
df = df.sort_values(by='date')
df.head(3)

,PREM_ID,ACCT_ID,CUSTOMER_NAME,first_name,last_name,CELL_PHONE,EMAIL,PREM_ADDRESS1,PREM_ADDRESS2,PREM_CITY,...,MAIL_STATE,MAIL_ZIP,CARE,TOTAL_DUE,PAST_DUE,LAST_BILL_DT,CAREpropensity,AMP Enrolled,ESA Treated,date
83808,6465077597,7272026044,"T***,W****",T***,W****,(650) 722-6449,D**********@HOTMAIL.COM,362 NORTHUMBERLAND AVE,NaN,REDWOOD CITY,...,CA,940613838,1,263.59,146.84,44608.0,2,N,N,2022-02-01
89668,5459669081,5011779561,"N****,M*****",N****,M*****,(415) 350-2993,NaN,72 LYELL ST,NaN,SAN FRANCISCO,...,CA,941121424,1,299.89,156.63,44593.0,4,N,N,2022-02-01
89669,9626784110,3477083841,"H***,J****",H***,J****,(415) 324-7111,J**********@GMAIL.COM,281 DRAKE AVE,NaN,SAUSALITO,...,CA,949651256,1,105.49,3.83,44610.0,Unknown,N,N,2022-02-01


### Write file to Excel

In [9]:
# Write to excel, including a label for column[0] = "index"
# df.to_excel('C:\\Users\\Robert.Jones\\Documents\\LiHEAP\\transformed_data\\propensity_transformed.xlsx',index=True, index_label="index")